## LoRaWAN Regression to predict Distances

In [18]:
!pip install scikit-optimize 

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.3/100.3 kB 72.1 kB/s eta 0:00:00a 0:00:01


In [20]:
import math
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy.optimize import minimize
from sklearn.cross_decomposition import PLSRegression
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import LinearRegression
import sklearn.metrics as metrics
from sklearn.model_selection import train_test_split, RandomizedSearchCV, GridSearchCV
from skopt.searchcv import BayesSearchCV
from sklearn.neural_network import MLPRegressor
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler, PolynomialFeatures, FunctionTransformer
import haversine as hs
from haversine import Unit

## Preprocessing

In [8]:
label = LabelEncoder()
data = pd.read_pickle("../../helium-api-wrapper/data/challenges.pkl")
data["datarate"] = label.fit_transform(data["datarate"])
# data["valid"] = label.fit_transform(data["valid"])
# NUM_WITTNESSES = int(data.index[-1]/data["key"].values[-1])

for i in range(len(data)):
    data["distance"][i] = hs.haversine(
        (data["challengee_lat"].iloc[i], data["challengee_lng"].iloc[i]),
        (data["witness_lat"].iloc[i], data["witness_lng"].iloc[i]),
        unit=Unit.METERS
    )
    
data["distance"]
data.drop(["challengee", "witness", "hash"], axis=1, inplace=True)

/var/folders/1v/z5r7qqm173d03z8jk9x6x9xm0000gn/T/ipykernel_5722/2064244353.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["distance"][i] = hs.haversine(
/var/folders/1v/z5r7qqm173d03z8jk9x6x9xm0000gn/T/ipykernel_5722/2064244353.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["distance"][i] = hs.haversine(
/var/folders/1v/z5r7qqm173d03z8jk9x6x9xm0000gn/T/ipykernel_5722/2064244353.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-ve

In [5]:
def regression_results(y_test, y_pred):

    # Regression metrics
    explained_variance=metrics.explained_variance_score(y_test, y_pred)
    mean_absolute_error=metrics.mean_absolute_error(y_test, y_pred) 
    mse=metrics.mean_squared_error(y_test, y_pred) 
    median_absolute_error=metrics.median_absolute_error(y_test, y_pred)
    r2=metrics.r2_score(y_test, y_pred)

    print('   explained_variance: ', round(explained_variance,4))    
    print('   r2: ', round(r2,4))
    print('   MAE: ', round(mean_absolute_error,4))
    print('   MSE: ', round(mse,4))
    print('   RMSE: ', round(np.sqrt(mse),4))

In [7]:
data = data.drop(data[(data["snr"].values < 4) & (data["snr"].values > -7.5)].index)
data = data.drop(data[data["signal"].values < -110].index)
data = data.drop(data[data["distance"].values == 0].index)
data = data.drop(data[data["distance"].values > 16000].index)
data

,challengee_lat,challengee_lng,witness_lat,witness_lng,signal,snr,datarate,is_valid,time,distance


In [9]:
data.describe()

,challengee_lat,challengee_lng,witness_lat,witness_lng,signal,snr,datarate,time,distance
count,1554.000000,1554.000000,1554.000000,1554.000000,1554.000000,1554.000000,1554.000000,1.554000e+03,1554.000000
mean,39.372278,-33.367122,39.362182,-33.391359,-126.014801,-12.009653,0.536680,1.669735e+09,6316.196903
std,15.074687,64.995864,15.072108,65.022357,12.855396,7.433725,0.498813,2.854986e+01,3588.042175
min,-38.193651,-149.204075,-38.198782,-149.315706,-318.000000,-24.000000,0.000000,1.669735e+09,618.517865
25%,35.275218,-87.886427,35.156265,-87.911284,-134.000000,-18.000000,0.000000,1.669735e+09,618.517865
50%,41.034064,-8.095726,41.015070,-8.296131,-127.000000,-12.800000,1.000000,1.669735e+09,8515.040873
75%,48.738756,13.658453,48.732514,13.705444,-121.000000,-8.200000,1.000000,1.669735e+09,8641.468565
max,64.829461,153.521464,64.847518,153.550709,-12.000000,11.800000,1.000000,1.669735e+09,8641.468565


In [10]:
y = data["distance"]
X = data.drop(["distance", "challengee_lat", "challengee_lng"], axis=1)
X

,witness_lat,witness_lng,signal,snr,datarate,is_valid,time
0,50.690230,14.513927,-139,-20.500000,0,True,1669735444
1,50.689996,14.545720,-132,-19.000000,0,True,1669735444
2,50.761089,14.526223,-99,5.000000,0,True,1669735444
0,53.064917,8.873005,-139,-22.200001,0,True,1669735444
0,47.609244,-122.298150,-127,-13.200000,1,True,1669735444
...,...,...,...,...,...,...,...
2,40.001315,-105.029165,-123,-9.800000,1,True,1669735383
0,48.879827,2.243961,-138,-22.500000,0,True,1669735383
1,48.845759,2.233865,-133,-18.200001,0,True,1669735383
2,48.842831,2.231666,-132,-21.200001,0,True,1669735383


In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=10011)

## Random-Forest Regressor

In [12]:
random_forest_regression = RandomForestRegressor(n_estimators=100, max_depth=20)
random_forest_regression.fit(X_train, y_train)
random_forest_regression_yPred = random_forest_regression.predict(X_test)

regression_results(y_test, random_forest_regression_yPred)

   explained_variance:  -0.0787
   r2:  -0.0793
   MAE:  3127.8707
   MSE:  14433421.0068
   RMSE:  3799.1342


## Randomized Search

In [14]:
rf  = Pipeline(
    steps=[
        ("scaler", StandardScaler()),
        ("estimator", RandomForestRegressor(warm_start=True, oob_score=True))
    ]
)

rgrid = {
    "estimator__n_estimators": np.arange(50,500,10),
    "estimator__criterion": ["squared_error", "absolute_error", "poisson"],
    "estimator__min_samples_split":np.arange(1, 50),
    "estimator__min_samples_leaf":np.arange(1, 50),
    "estimator__max_features": ["sqrt", "log2"],
    "estimator__max_depth": np.arange(2, 20)
}


opt_rf = RandomizedSearchCV(rf, rgrid, cv = 5, scoring="neg_root_mean_squared_error", random_state=10011)
opt_rf.fit(X_train, y_train)
yrf_pred = opt_rf.predict(X_test)
regression_results(y_test, yrf_pred)
print(opt_rf.best_params_)

/Users/niklas/.pyenv/versions/3.8.12/envs/3.8_DataScience/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:372: FitFailedWarning: 
5 fits failed out of a total of 50.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
5 fits failed with the following error:
Traceback (most recent call last):
  File "/Users/niklas/.pyenv/versions/3.8.12/envs/3.8_DataScience/lib/python3.8/site-packages/sklearn/model_selection/_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/niklas/.pyenv/versions/3.8.12/envs/3.8_DataScience/lib/python3.8/site-packages/sklearn/pipeline.py", line 394, in fit
    self._final_estimator.fit(Xt, y, **fit_params_last_step)
  File "/Users/niklas/.pyenv/ver

   explained_variance:  0.0677
   r2:  0.0667
   MAE:  3103.0773
   MSE:  12481509.7014
   RMSE:  3532.918
{'estimator__n_estimators': 400, 'estimator__min_samples_split': 6, 'estimator__min_samples_leaf': 42, 'estimator__max_features': 'sqrt', 'estimator__max_depth': 12, 'estimator__criterion': 'squared_error'}


## Gridsearch

In [15]:
rf  = Pipeline(
    steps=[
        ("scaler", StandardScaler()),
        ("estimator", RandomForestRegressor(warm_start=True, oob_score=True))
    ]
)

rgrid = {
    "estimator__n_estimators": np.arange(50,200,10),
    "estimator__max_depth": np.arange(2, 20)
}


opt_rf = GridSearchCV(rf, rgrid, cv = 5, scoring="neg_root_mean_squared_error")
opt_rf.fit(X_train, y_train)
yrf_pred = opt_rf.predict(X_test)
regression_results(y_test, yrf_pred)
print(opt_rf.best_params_)

   explained_variance:  0.0565
   r2:  0.0555
   MAE:  3095.8852
   MSE:  12630952.2838
   RMSE:  3554.0051
{'estimator__max_depth': 3, 'estimator__n_estimators': 50}


## BayesSearch CV

In [23]:
rf  = Pipeline(
    steps=[
        ("scaler", StandardScaler()),
        ("estimator", RandomForestRegressor(warm_start=True, oob_score=True))
    ]
)

rgrid = {
    "estimator__n_estimators": np.arange(50,500,10),
    "estimator__criterion": ["squared_error", "absolute_error", "poisson"],
    "estimator__min_samples_split":np.arange(1, 50),
    "estimator__min_samples_leaf":np.arange(5, 50),
    "estimator__max_features": ["sqrt", "log2"],
    "estimator__max_depth": np.arange(2, 20)
}

bayes_rf = BayesSearchCV(rf, rgrid, cv = 5, n_jobs=1, scoring="neg_root_mean_squared_error")
bayes_rf.fit(X_train, y_train)
yrf_pred = bayes_rf.predict(X_test)
regression_results(y_test, yrf_pred)
print(bayes_rf.best_params_)

   explained_variance:  0.0694
   r2:  0.0685
   MAE:  3107.3978
   MSE:  12457669.183
   RMSE:  3529.5423
OrderedDict([('estimator__criterion', 'squared_error'), ('estimator__max_depth', 13), ('estimator__max_features', 'log2'), ('estimator__min_samples_leaf', 47), ('estimator__min_samples_split', 28), ('estimator__n_estimators', 90)])


## Gradient Boosting Regressor with Randomized Search Tuning

In [24]:
pipeline = Pipeline(
    steps=[
        ("scaler", StandardScaler()),
        ("estimator", GradientBoostingRegressor())
    ]
)

grid = {
    "estimator__n_estimators": np.arange(10,500,10),
    "estimator__learning_rate": np.linspace(1e-3, 0.5,50),
    "estimator__max_depth": np.arange(1,15), 
    "estimator__min_samples_split": np.linspace(1e-7, 0.99,100),
    "estimator__max_features": ["sqrt", "log2"],
    "estimator__loss": ["squared_error", "absolute_error", "huber", "quantile"]
}

gbt_grid = RandomizedSearchCV(pipeline, grid, cv=5,scoring="neg_root_mean_squared_error", random_state=10011)

gbt_grid.fit(X_train, y_train)
y_pred = gbt_grid.predict(X_test)
regression_results(y_test, y_pred)
print(gbt_grid.best_params_)

   explained_variance:  0.0443
   r2:  0.043
   MAE:  3077.5319
   MSE:  12798616.5478
   RMSE:  3577.5154
{'estimator__n_estimators': 160, 'estimator__min_samples_split': 0.5000000494949495, 'estimator__max_features': 'log2', 'estimator__max_depth': 6, 'estimator__loss': 'squared_error', 'estimator__learning_rate': 0.04173469387755102}


## BayesSearch CV

In [25]:
gbt  = Pipeline(
    steps=[
        ("scaler", StandardScaler()),
        ("estimator", GradientBoostingRegressor())
    ]
)

gbt_grid_b = {
    "estimator__n_estimators": np.arange(50,500,10),
    "estimator__criterion": ["squared_error", "absolute_error", "poisson"],
    "estimator__min_samples_split":np.arange(1, 50),
    "estimator__min_samples_leaf":np.arange(1, 50),
    "estimator__max_features": ["sqrt", "log2"],
    "estimator__max_depth": np.arange(2, 20)
}

bayes_gbt = BayesSearchCV(gbt, rgrid, cv = 5, n_jobs=1, scoring="neg_root_mean_squared_error")
bayes_gbt.fit(X_train, y_train)
yrf_pred = bayes_gbt.predict(X_test)
regression_results(y_test, yrf_pred)
print(bayes_gbt.best_params_)

/Users/niklas/.pyenv/versions/3.8.12/envs/3.8_DataScience/lib/python3.8/site-packages/sklearn/ensemble/_gb.py:1854: FutureWarning: criterion='mae' was deprecated in version 0.24 and will be removed in version 1.1 (renaming of 0.26). The correct way of minimizing the absolute error is to use  loss='absolute_error' instead.
  warnings.warn(
/Users/niklas/.pyenv/versions/3.8.12/envs/3.8_DataScience/lib/python3.8/site-packages/sklearn/ensemble/_gb.py:1854: FutureWarning: criterion='mae' was deprecated in version 0.24 and will be removed in version 1.1 (renaming of 0.26). The correct way of minimizing the absolute error is to use  loss='absolute_error' instead.
  warnings.warn(
/Users/niklas/.pyenv/versions/3.8.12/envs/3.8_DataScience/lib/python3.8/site-packages/sklearn/ensemble/_gb.py:1854: FutureWarning: criterion='mae' was deprecated in version 0.24 and will be removed in version 1.1 (renaming of 0.26). The correct way of minimizing the absolute error is to use  loss='absolute_error' ins

   explained_variance:  0.0
   r2:  -0.0015
   MAE:  3298.3085
   MSE:  13393846.9044
   RMSE:  3659.7605
OrderedDict([('estimator__criterion', 'poisson'), ('estimator__max_depth', 17), ('estimator__max_features', 'sqrt'), ('estimator__min_samples_leaf', 13), ('estimator__min_samples_split', 14), ('estimator__n_estimators', 320)])
